# Amazon Interview Question
Write a query to provide the date for nth occurence of Sunday in future from given date

In [0]:
WITH params AS (
  SELECT DATE('2022-01-01') AS start_date, 3 AS n
),
calc AS (
  SELECT 
    start_date,
    n,
    DAYOFWEEK(start_date) AS weekday_num
  FROM params
),
next_sunday AS (
  SELECT
    start_date,
    n,
    CASE 
      WHEN DAYOFWEEK(start_date) = 1 THEN 0  -- Already Sunday
      ELSE 8 - DAYOFWEEK(start_date)
    END AS days_to_next_sunday
  FROM calc
)
SELECT 
  start_date,
  n,
  DATE_ADD(start_date, days_to_next_sunday + ((n - 1) * 7)) AS nth_sunday_date
FROM next_sunday;

In [0]:
%python
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Create Spark session (already available in Databricks)
spark = SparkSession.builder.getOrCreate()

# Input parameters
start_date = '2022-01-01'
n = 3

# Create initial DataFrame with params
df = spark.createDataFrame([(start_date, n)], ["start_date", "n"]) \
           .withColumn("start_date", F.to_date("start_date"))

# Step 1️⃣: Calculate weekday number (Sunday = 1, Saturday = 7)
df = df.withColumn("weekday_num", F.dayofweek("start_date"))

# Step 2️⃣: Calculate days to next Sunday
df = df.withColumn(
    "days_to_next_sunday",
    F.when(F.col("weekday_num") == 1, F.lit(0))  # Already Sunday
     .otherwise(8 - F.col("weekday_num"))         # Days until next Sunday
)

# Step 3️⃣: Add (n-1)*7 + days_to_next_sunday
df = df.withColumn(
    "nth_sunday_date",
    F.date_add(
        "start_date",
        F.col("days_to_next_sunday") + ((F.col("n") - 1) * 7).cast("int")
    )
)

# Step 4️⃣: Display final result
display(df.select("start_date", "n", "nth_sunday_date"))
